# Pré processamento dos dados

- [Link dos dados](https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv)
- baixar o csv e colocar na pasta `./data`

In [1]:
import pandas as pd
import re

In [10]:
file_path = 'data/20250401_atividade_economica.csv'
df = pd.read_csv(file_path, sep=';')

In [11]:
df.head()

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL,CNAE,DATA_INICIO_ATIVIDADE,NATUREZA_JURIDICA,PORTE_EMPRESA,AREA_UTILIZADA,IND_SIMPLES,IND_MEI,...,FORMA_ATUACAO,DESC_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME_BAIRRO,NOME,NOME_FANTASIA,CNPJ,GEOMETRIA
0,1006,9521500.0,REPARAÇÃO E MANUTENÇÃO DE EQUIPAMENTOS ELETROE...,"3314707, 4322302, 4649404, 4651601, 4669999, 4...",19-04-1994,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,407.0,S,N,...,ESTABELECIMENTO FIXO,RUA,NIQUELINA,874,"GALPAO,",SANTA EFIGENIA,ORLANDO SERVICE LTDA,NaN,97502553000184,POINT (613854.46 7797085.18)
1,1007,4755502.0,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,"4755502, 8599604",12-04-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,20.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS GUAJAJARAS,910,SALA 612,CENTRO,TARGET LTDA,NaN,70950258000134,POINT (610658.10 7796519.70)
2,1008,4520001.0,SERVIÇOS DE MANUTENÇÃO E REPARAÇÃO MECÂNICA DE...,"4520001, 4520003, 4530703",16-11-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,260.0,S,N,...,ESTABELECIMENTO FIXO,RUA,CONSELHEIRO LAFAIETE,284,NaN,SAGRADA FAMILIA,HUDSON CAR LTDA,HUDSON CAR,731235000153,POINT (612523.34 7798131.15)
3,1009,4616800.0,REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCI...,4616800,16-11-1992,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,1300.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS TIMBIRAS,2352,SALA 01,LOURDES,SILHUETA REPRESENTACAO DE ROUPAS LTDA,GARMENT,68519362000127,POINT (610394.12 7796444.35)
4,1010,5822101.0,EDIÇÃO INTEGRADA À IMPRESSÃO DE JORNAIS DIÁRIOS,5822101,01-06-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,30.0,N,N,...,ESTABELECIMENTO FIXO,RUA,SAO FELICIANO,73,NaN,CAICARAS,JORNAL CAICARAS LTDA,NaN,71078968000189,POINT (607875.47 7799877.16)


## Filtragem por bares e restaurantes

In [3]:
words = ['bar', 'bares', 'restaurante', 'restaurantes']

Constrói a expressão regex que da match em qualquer uma das palavras, se ela aparecer como uma palavra separada por espaços ou pontuações das outras:

In [4]:
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b'
pattern

'\\b(?:bar|bares|restaurante|restaurantes)\\b'

In [5]:
filtered_table = df[['CNAE_PRINCIPAL', 'DESCRICAO_CNAE_PRINCIPAL']].drop_duplicates()
filtered_table = filtered_table[filtered_table['DESCRICAO_CNAE_PRINCIPAL'].str.contains(pattern, case=False, na=False)]

for value in filtered_table['DESCRICAO_CNAE_PRINCIPAL']:
    print(value)
filtered_table

RESTAURANTES E SIMILARES
BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS EM SERVIR BEBIDAS, SEM ENTRETENIMENTO
BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS EM SERVIR BEBIDAS, COM ENTRETENIMENTO 


,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL
17,5611201.0,RESTAURANTES E SIMILARES
679,5611204.0,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...
3079,5611205.0,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...


In [6]:
CNAES = [int(cnae) for cnae in filtered_table['CNAE_PRINCIPAL']]
CNAES

[5611201, 5611204, 5611205]

In [7]:
filtered_df = df[df['CNAE_PRINCIPAL'].isin(CNAES)].copy()

filtered_df['CNAE_PRINCIPAL'].value_counts()

CNAE_PRINCIPAL
5611201.0    8550
5611204.0    3570
5611205.0    1505
Name: count, dtype: int64

## Formatação do Endereço

In [8]:
filtered_df['ENDERECO_FORMATADO'] = (
  filtered_df['DESC_LOGRADOURO'] + ' ' +
  filtered_df['NOME_LOGRADOURO'] + ', ' +
  filtered_df['NUMERO_IMOVEL'].astype(str) +
  filtered_df['COMPLEMENTO'].apply(lambda x: f", {x}" if pd.notnull(x) else "")
)

filtered_df[['ENDERECO_FORMATADO']].head()

,ENDERECO_FORMATADO
17,"RUA DESEMBARGADOR REIS ALVES, 90, ANDAR: 2,"
19,"RUA LUIZ PONGELUPE, 290, C"
29,"AVE RESSACA, 118, LOJA 04"
172,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135"
363,"AVE CRISTIANO MACHADO, 1950, LOJA 36"


## Tabela Final

In [9]:
filtered_table = filtered_df[['DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO_FORMATADO', 'NOME_FANTASIA']].drop_duplicates()

filtered_table = filtered_table[filtered_table['NOME_FANTASIA'].notna()]

filtered_table.head()

,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_FORMATADO,NOME_FANTASIA
19,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290, C",PIZZARIA E CHURRASCARIA VARANDA
29,02-05-1994,NÃO,"AVE RESSACA, 118, LOJA 04",RABBIT BURGER
172,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135",TATU REI DO ANGU A BAHIANA
363,01-10-1993,SIM,"AVE CRISTIANO MACHADO, 1950, LOJA 36",ESPACO DA FEIJOADA
662,16-09-2011,SIM,"RUA DOS OTONI, 222, LETRA: A,",RESTAURANTE NOVO SABOR


In [14]:
filtered_table.shape

(8663, 4)

In [12]:
filtered_table.to_csv('data/dados.csv', index=False)